In [1]:
import pandas as pd
import os
import numpy as np
from glob import glob

In [2]:
f1 = glob('NIA*.xlsx')[0]

In [3]:
df = pd.read_excel(f1)

In [4]:
df2 = df[df['Status']=='Smartform submitted by Client']

In [5]:
df2= df2.sort_values(by=['UPC','Item ID','Item Description'])

In [6]:
df2['len_externalcode']=df['UPC'].astype(str).str.len()

In [7]:
df2['T&F_UPC'] = df2['len_externalcode'].apply(lambda x:'True' if x>10 else 'False')

In [8]:
df2['FS1'] = np.where(df2['T&F_UPC'] == 'False', 'Rejected', 'Accepted')

In [9]:
df2['check1']=df2['UPC'].shift(-1)==df2['UPC']

In [10]:
df2['check1']=df2['check1'].astype(str)

In [11]:
def set_rs(df2):
    if(df2['FS1']=='Rejected')|(df2['check1']=='True'):
        return "Rejected"
    else:
        return "Accepted"

In [12]:
df2= df2.assign(result1=df2.apply(set_rs, axis=1))

In [13]:
df3=pd.read_excel('TemplateName & Coder Info.xlsx')

In [14]:
df4 = df2.merge(df3,left_on="Template Name",right_on="Template Name",
                   how="left")

In [15]:
df4['result1']=df4['result1'].astype(str)

In [16]:
df4['final_allocation']=np.where(df4['result1'] == "Rejected", 'Dhawal', df4['Allocation'])

In [17]:
df4.to_excel("Final_Allocation.xlsx")

In [18]:
text_file = open("SourceCode.txt", "w")

text_file.write("""import pandas as pd
import os
import numpy as np
from glob import glob
f1 = glob('NIA allocation*.xlsx')[0]
df = pd.read_excel(f1)
df2 = df[df['Status']=='Smartform submitted by Client']
df2= df2.sort_values(by=['UPC','Item ID','Item Description'])
df2['len_externalcode']=df['UPC'].astype(str).str.len()
df2['T&F_UPC'] = df2['len_externalcode'].apply(lambda x:'True' if x>=10 else 'False')
df2['FS1'] = np.where(df2['T&F_UPC'] == 'False', 'Rejected', 'Accepted')
df2['check1']=df2['UPC'].shift(-1)==df2['UPC']
df2['check1']=df2['check1'].astype(str)
def set_rs(df2):
    if(df2['FS1']=='Rejected')|(df2['check1']=='True'):
        return "Rejected"
    else:
        return "Accepted"

df2= df2.assign(result1=df2.apply(set_rs, axis=1))
df3=pd.read_excel('TemplateName & Coder Info.xlsx')
df4 = df2.merge(df3,left_on="Template Name",
                   right_on="Template Name", how="left")
df4['result1']=df4['result1'].astype(str)
df4['final_allocation']=np.where(df4['result1'] == "Rejected", 'Dhawal', df4['Coder'])
df4.to_excel("Final_Allocation.xlsx")

""")

text_file.close()

In [19]:
#df5= df4[['Item ID','final_allocation']]

In [20]:
#df5.to_excel("allocationfile_Aug06.xlsx")